In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
import gc
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import PIL
from PIL import ImageDraw
from sklearn.model_selection import StratifiedKFold

from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

from keras.applications import Xception
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras import layers, models, optimizers

Using TensorFlow backend.


In [3]:
warnings.filterwarnings(action='ignore')

K.image_data_format()

'channels_last'

In [4]:
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
# DATA_PATH = '/gdrive/My Drive/Colab Notebooks'

In [4]:
IMAGE_SIZE = 299 #Image_size는 사용하는 pretained 모델 별 추천하는 이미지 사이즈를 주기 위해
BATCH_SIZE = 16
EPOCHS = 100

In [5]:
import os
import numpy as np
import pandas as pd

In [22]:
# 이미지 폴더 경로
TRAIN_IMG_PATH = os.path.join(DATA_PATH, 'final_train_pill')
VAL_IMG_PATH = os.path.join(DATA_PATH,'img_val_crop')

# CSV 파일 경로
df_train = pd.read_csv(os.path.join(DATA_PATH, 'df_final_pill_image.csv'))
df_val = pd.read_csv(os.path.join(DATA_PATH, 'df_val_final.csv'))
df_class = pd.read_csv(os.path.join(DATA_PATH, 'df_class.csv'))

FileNotFoundError: ignored

In [0]:
def del_index(a):
    if '.DS_Store' in a:
        del a[a.index('.DS_Store')]
        print('.DS_Store 삭제')
    if '.ipynb_checkpoints' in a:
        del a[a.index('.ipynb_checkpoints')]
        print('checkpoints 삭제')

In [0]:
train_list = os.listdir(TRAIN_IMG_PATH)
val_list = os.listdir(VAL_IMG_PATH)

In [0]:
del_index(train_list)
del_index(val_list)

In [12]:
# Data 누락 체크
if set(list(df_train.img)) == set(train_list) :
    print("Train file 누락 없음!")
else : 
    print("Train file 누락")

if set(list(df_val.img)) == set(val_list) :
    print("val file 누락 없음!")
else : 
    print("val file 누락")


Train file 누락
val file 누락 없음!


In [14]:
# Data 갯수
#print("Number of Train Data : {}".format(df_train.shape[0]))
#print("Number of Test Data : {}".format(df_test.shape[0]))
#print("타겟 클래스 총 갯수 : {}".format(df_class.shape[0]))
print("Train Data의 타겟 종류 갯수 : {}".format(df_train['class'].nunique()))

cntEachClass = df_train["class"].value_counts(ascending=False)
# Data 정보
print("Class with most count : {}".format(cntEachClass.index[0]))
print("Most Count : {}".format(cntEachClass.max()))
print("Class with fewest count : {}".format(cntEachClass.min()))
print("Mean : {}".format(cntEachClass.mean()))

Train Data의 타겟 종류 갯수 : 91
Class with most count : 655500320
Most Count : 22
Class with fewest count : 6
Mean : 13.934065934065934


In [15]:
from sklearn.model_selection import train_test_split

df_train["class"] = df_train["class"].astype('str')
df_val["class"] = df_val["class"].astype('str')
df_train = df_train[['img','class']]
df_val = df_val[['img','class']]

X_train = df_train.sample(frac=1).reset_index(drop=True)
X_val = df_val.sample(frac=1).reset_index(drop=True)

print(X_train.shape)
print(X_val.shape)
#print(df_test.shape)

(1268, 2)
(179, 2)


In [6]:
model_path = './model/'
if not os.path.exists(model_path):
    os.mkdir(model_path)

In [7]:
def get_callback(model_name, patient):
    ES = EarlyStopping(monitor = 'val_loss',
                      patience = patient,
                      mode = 'min',
                      verbose = 1)
    RR = ReduceLROnPlateau(monitor = 'val_loss',
                          factor = 0.5,
                          patience = patient / 2,
                          min_lr = 0.000001,
                          verbose = 1,
                          mode = 'min')
    MC = ModelCheckpoint(filepath = filepath,
                        monitor = 'val_loss',
                        verbose = 1, 
                        save_best_only = True,
                        mode ='min')
    return [ES,RR,MC]

In [8]:
def get_model(model_name, iamge_size):
    base_model = Xception(weights='imagenet', include_top=False)
    #base_model.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(89, activation='softmax'))
    model.summary()

    optimizer = optimizers.RMSprop(lr=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

    return model

In [9]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.models import Sequential, Model
from keras.layers import Dense,Dropout,Flatten,Activation, Conv2D, GlobalAveragePooling2D
# Parameter
#img_size = (224, 224)
#nb_train_samples = len(X_train)
#nb_validation_samples = len(X_val)
#nb_test_samples = len(df_test)
#epochs = 20
#batch_size = 32

# Define Generator config
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True, 
    vertical_flip = False,
    zoom_range=0.2,
    fill_mode = 'nearest'
    #preprocessing_function=preprocess_input)
)

val_datagen = ImageDataGenerator(rescale = 1./255
#preprocessing_function=preprocess_input)
                                )
test_datagen = ImageDataGenerator(rescale = 1./255
#preprocessing_function=preprocess_input
                                )
# Make Generator

train_generator = train_datagen.flow_from_dataframe(
    dataframe=X_train, 
    directory=TRAIN_IMG_PATH,
    x_col = 'img',
    y_col = 'class',
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=16,
    seed=42,
    shuffle = True)
    
valid_generator = val_datagen.flow_from_dataframe(
    dataframe=X_val, 
    directory=VAL_IMG_PATH,
    x_col = 'img',
    y_col = 'class',
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=16,
    shuffle = True)


NameError: name 'X_train' is not defined

In [10]:
model_xception = get_model(Xception, IMAGE_SIZE)

Instructions for updating:
Colocations handled automatically by placer.


39256064/83683744 [=============>................] - ETA: 10 - ETA: 17:2 - ETA: 22:2 - ETA: 10:2 - ETA: 13:0 - ETA: 12:0 - ETA: 10:4 - ETA: 10:2 - ETA: 11:4 - ETA: 11:0 - ETA: 10:3 - ETA: 10:5 - ETA: 10:5 - ETA: 10:3 - ETA: 11:0 - ETA: 10:3 - ETA: 10:2 - ETA: 10:3 - ETA: 10:3 - ETA: 10:2 - ETA: 10:5 - ETA: 10:3 - ETA: 10:3 - ETA: 10:3 - ETA: 10:3 - ETA: 10:2 - ETA: 10:5 - ETA: 10:3 - ETA: 10:3 - ETA: 10:3 - ETA: 10:2 - ETA: 10:2 - ETA: 10:2 - ETA: 10:1 - ETA: 10:1 - ETA: 10:1 - ETA: 10:1 - ETA: 10:1 - ETA: 10:2 - ETA: 10:1 - ETA: 10:1 - ETA: 10:1 - ETA: 10:2 - ETA: 10:1 - ETA: 10:1 - ETA: 10:1 - ETA: 10:1 - ETA: 10:1 - ETA: 10:0 - ETA: 10:1 - ETA: 10:1 - ETA: 10:1 - ETA: 10:2 - ETA: 10:2 - ETA: 10:2 - ETA: 10:2 - ETA: 10:3 - ETA: 10:2 - ETA: 10:2 - ETA: 10:2 - ETA: 10:2 - ETA: 10:3 - ETA: 10:2 - ETA: 10:3 - ETA: 10:3 - ETA: 10:3 - ETA: 10:2 - ETA: 10:3 - ETA: 10:3 - ETA: 10:3 - ETA: 10:3 - ETA: 10:2 - ETA: 10:3 - ETA: 10:2 - ETA: 10:3 - ETA: 10:3 - ETA: 10:2 - ETA: 10:2 - ETA: 10:2 - E

78462976/83683744 [===========================>..] - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1:

83689472/83683744 [==============================] - ETA: 22 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 

In [ ]:
filepath = "my_Xception_model_{val_acc:.2f}_{val_loss:.4f}.h5"

In [0]:
history = model_xception.fit_generator(train_generator,
                                    steps_per_epoch = len(X_train.index) / BATCH_SIZE,
                                    epochs = EPOCHS,
                                    validation_data = valid_generator,
                                    validation_steps = len(X_val.index) / BATCH_SIZE,
                                    verbose = 1,
                                    shuffle = False,
                                    callbacks = get_callback(model_xception, 4)
                                    )
gc.collect()

Epoch 1/100
35/34 [==============================] - 65s 2s/step - loss: 4.4875 - acc: 0.0143 - val_loss: 4.4409 - val_acc: 0.0670

Epoch 00001: val_loss improved from inf to 4.44092, saving model to my_Xception_model_0.07_4.4409.h5
Epoch 2/100
35/34 [==============================] - 51s 1s/step - loss: 4.2390 - acc: 0.0714 - val_loss: 4.2553 - val_acc: 0.1285

Epoch 00002: val_loss improved from 4.44092 to 4.25531, saving model to my_Xception_model_0.13_4.2553.h5
Epoch 3/100
35/34 [==============================] - 51s 1s/step - loss: 3.7676 - acc: 0.1625 - val_loss: 3.8330 - val_acc: 0.1676

Epoch 00003: val_loss improved from 4.25531 to 3.83305, saving model to my_Xception_model_0.17_3.8330.h5
Epoch 4/100
35/34 [==============================] - 51s 1s/step - loss: 3.2747 - acc: 0.2572 - val_loss: 3.2999 - val_acc: 0.3128

Epoch 00004: val_loss improved from 3.83305 to 3.29995, saving model to my_Xception_model_0.31_3.2999.h5
Epoch 5/100
35/34 [==============================] - 51s

85

In [0]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [0]:
model_list = sorted([i for i in os.listdir() if "my_" in i])

In [0]:
model_list

['my_Xception_model_0.06_4.4046.h5',
 'my_Xception_model_0.07_4.4409.h5',
 'my_Xception_model_0.13_4.2553.h5',
 'my_Xception_model_0.15_4.1471.h5',
 'my_Xception_model_0.17_3.8330.h5',
 'my_Xception_model_0.21_3.6704.h5',
 'my_Xception_model_0.28_3.1420.h5',
 'my_Xception_model_0.31_3.2999.h5',
 'my_Xception_model_0.35_2.8078.h5',
 'my_Xception_model_0.39_2.2241.h5',
 'my_Xception_model_0.41_2.6017.h5',
 'my_Xception_model_0.46_2.2874.h5',
 'my_Xception_model_0.51_1.8860.h5',
 'my_Xception_model_0.54_1.8257.h5',
 'my_Xception_model_0.58_1.6279.h5',
 'my_Xception_model_0.61_1.4835.h5',
 'my_Xception_model_0.65_1.2882.h5',
 'my_Xception_model_0.66_1.2153.h5',
 'my_Xception_model_0.66_1.4271.h5',
 'my_Xception_model_0.67_1.1803.h5',
 'my_Xception_model_0.67_1.2234.h5',
 'my_Xception_model_0.69_0.9279.h5',
 'my_Xception_model_0.73_0.9303.h5',
 'my_Xception_model_0.73_0.9646.h5',
 'my_Xception_model_0.73_1.0084.h5',
 'my_Xception_model_0.74_1.1458.h5',
 'my_Xception_model_0.75_0.9829.h5',
 

In [0]:
model_xception.save_weights('/gdrive/My Drive/Colab Notebooks/model/model_xception_best.h5')

In [11]:
model_list = os.listdir('./model/')
model_list

['my_DenseNet121_model_0.82_0.7317.h5',
 'my_ResNet50_model_0.87_0.7278.h5',
 'my_Xception_model_0.88_0.6905.h5',
 'my_Xception_model_1_0.82_0.5864.h5',
 'my_Xception_model_2_0.84_0.6394.h5']

In [13]:
model_xception.load_weights('./model/' + model_list[-1])

In [0]:
from keras.applications.resnet50 import ResNet50
def get_RESNET_model(model_name, iamge_size):
    base_model = ResNet50(weights='imagenet', include_top=False)
    #base_model.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(89, activation='softmax'))
    model.summary()

    optimizer = optimizers.RMSprop(lr=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

    return model

In [0]:
model_ResNet = get_model(ResNet50, IMAGE_SIZE)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
global_average_pooling2d_5 ( (None, 2048)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_9 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dropout_10 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 89)                91225     
Total para

In [0]:
filepath = "my_ResNet50_model_{val_acc:.2f}_{val_loss:.4f}.h5"

history = model_ResNet.fit_generator(train_generator,
                                    steps_per_epoch = len(X_train.index) / BATCH_SIZE,
                                    epochs = EPOCHS,
                                    validation_data = valid_generator,
                                    validation_steps = len(X_val.index) / BATCH_SIZE,
                                    verbose = 1,
                                    shuffle = False,
                                    callbacks = get_callback(model_ResNet, 4)
                                    )
gc.collect()

Epoch 1/100
35/34 [==============================] - 68s 2s/step - loss: 4.4614 - acc: 0.0179 - val_loss: 4.4093 - val_acc: 0.0726

Epoch 00001: val_loss improved from inf to 4.40931, saving model to my_ResNet50_model_0.07_4.4093.h5
Epoch 2/100
35/34 [==============================] - 51s 1s/step - loss: 4.2298 - acc: 0.1000 - val_loss: 4.1934 - val_acc: 0.0894

Epoch 00002: val_loss improved from 4.40931 to 4.19342, saving model to my_ResNet50_model_0.09_4.1934.h5
Epoch 3/100
35/34 [==============================] - 51s 1s/step - loss: 3.7998 - acc: 0.2000 - val_loss: 3.8264 - val_acc: 0.2011

Epoch 00003: val_loss improved from 4.19342 to 3.82642, saving model to my_ResNet50_model_0.20_3.8264.h5
Epoch 4/100
35/34 [==============================] - 51s 1s/step - loss: 3.1747 - acc: 0.2876 - val_loss: 3.3294 - val_acc: 0.2626

Epoch 00004: val_loss improved from 3.82642 to 3.32944, saving model to my_ResNet50_model_0.26_3.3294.h5
Epoch 5/100
35/34 [==============================] - 51s

KeyboardInterrupt: ignored

In [0]:
model_ResNet.save_weights('/gdrive/My Drive/Colab Notebooks/model/ResNetbest.h5')

In [0]:
from keras.applications.densenet import DenseNet121
def get_DenseNet121_model(model_name, iamge_size):
    base_model = DenseNet121(weights='imagenet', include_top=False)
    #base_model.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(89, activation='softmax'))
    model.summary()

    optimizer = optimizers.RMSprop(lr=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

    return model

In [0]:
model_DenseNet121 = get_DenseNet121_model(DenseNet121, IMAGE_SIZE)

W0826 22:14:14.449270 140132891797376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



29089792/29084464 [==============================] - 0s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, None, None, 1024)  7037504   
_________________________________________________________________
global_average_pooling2d_7 ( (None, 1024)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dropout_13 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dropout_14 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_21 (D

In [0]:
filepath = "my_DenseNet121_model_{val_acc:.2f}_{val_loss:.4f}.h5"

history = model_DenseNet121.fit_generator(train_generator,
                                    steps_per_epoch = len(X_train.index) / BATCH_SIZE,
                                    epochs = EPOCHS,
                                    validation_data = valid_generator,
                                    validation_steps = len(X_val.index) / BATCH_SIZE,
                                    verbose = 1,
                                    shuffle = False,
                                    callbacks = get_callback(model_DenseNet121, 4)
                                    )
gc.collect()

Epoch 1/100
35/34 [==============================] - 76s 2s/step - loss: 4.5433 - acc: 0.0214 - val_loss: 4.4221 - val_acc: 0.0335

Epoch 00001: val_loss improved from inf to 4.42209, saving model to my_DenseNet121_model_0.03_4.4221.h5
Epoch 2/100
35/34 [==============================] - 31s 894ms/step - loss: 4.2003 - acc: 0.0857 - val_loss: 3.9879 - val_acc: 0.1508

Epoch 00002: val_loss improved from 4.42209 to 3.98785, saving model to my_DenseNet121_model_0.15_3.9879.h5
Epoch 3/100
35/34 [==============================] - 32s 900ms/step - loss: 3.7598 - acc: 0.1285 - val_loss: 3.7328 - val_acc: 0.1341

Epoch 00003: val_loss improved from 3.98785 to 3.73282, saving model to my_DenseNet121_model_0.13_3.7328.h5
Epoch 4/100
35/34 [==============================] - 32s 901ms/step - loss: 3.3173 - acc: 0.2197 - val_loss: 3.3520 - val_acc: 0.2011

Epoch 00004: val_loss improved from 3.73282 to 3.35202, saving model to my_DenseNet121_model_0.20_3.3520.h5
Epoch 5/100
35/34 [================

436

In [0]:
model_DenseNet121.save_weights('/gdrive/My Drive/Colab Notebooks/model/DenseNetbest.h5')

In [0]:
df_test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
TEST_IMG_PATH = os.path.join(DATA_PATH,'test_img')
if set(list(df_test.img)) == set(os.listdir(TEST_IMG_PATH)) :
    print("Test file 누락 없음!")
else : 
    print("Test file 누락")

In [0]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe = df_test,
    directory = TEST_IMG_PATH,
    x_col = 'img',
    y_col = None,
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    color_mode = 'rgb',
    class_mode = None,
    batch_size = BATCH_SIZE,
    shuffle = False)

In [0]:
%%time
test_generator.reset()
prediction = model_ResNet.predict_generator(
    generator = test_generator,
    steps = len(df_test)/BATCH_SIZE,
    verbose=1
)

1/0 [========================================] - 5s 5s/step
CPU times: user 4.61 s, sys: 197 ms, total: 4.81 s
Wall time: 4.57 s


In [0]:
predicted_class_indices=np.argmax(prediction, axis=1)

# Generator class dictionary mapping
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

#submission = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))
#submission["class"] = predictions
#submission.to_csv("submission.csv", index=False)
#submission

In [0]:
df_test['prediction'] = predictions

In [0]:
df_test

,class,img,prediction
0,650700520,0.jpeg,644700140
1,643504310,1.jpeg,643504310
2,645403740,2.jpeg,645403740
3,649900060,3.jpeg,641601880
4,644700140,4.jpeg,645301210
5,645301210,5.jpeg,647802630
6,644902890,6.jpeg,644900310
7,642202020,7.jpeg,642200220
8,641905450,8.jpeg,653600010
9,641602000,9.jpeg,643304610


In [0]:
model_list = sorted([i for i in os.listdir() if "my_" in i])

In [0]:
model_list

In [0]:
for i in model_list:
  i.save_weights('/gdrive/My Drive/Colab Notebooks/model/'+i )

AttributeError: ignored

In [19]:
from PIL import Image

In [20]:
def read_images(directory, resize_to=(299,299)):
    images = []
    
    for f in tqdm(os.listdir(directory)):         #tqdm: progress bar 표시
        
        #디렉토리 이름과 파일 이름을 연결.
        #directory: ./cats_dogs/train/
        #f: cat.0.jpg
        #path: ./cats_dogs/train/cat.0.jpg
        path = os.path.join(directory, f)
        
        im = Image.open(path)
        im = im.resize(resize_to)
        
        im = np.array(im) / 255.0   # 이미지 파일에 저장된 RGB 값은 0~255 사이로 저장되어 있음.ㅁ
        im = im.astype('float32')
        
        images.append(im)  # images에 im 추가
    
    return np.array(images)

In [34]:
images = read_images(directory='./test/')
images

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 18.26it/s]


array([[[[0.85490197, 0.8627451 , 0.8509804 ],
         [0.85490197, 0.8627451 , 0.8509804 ],
         [0.85490197, 0.8627451 , 0.8509804 ],
         ...,
         [0.84313726, 0.8509804 , 0.8392157 ],
         [0.84313726, 0.8509804 , 0.8392157 ],
         [0.84705883, 0.85490197, 0.84313726]],

        [[0.8509804 , 0.85882354, 0.84705883],
         [0.8509804 , 0.85882354, 0.84705883],
         [0.85490197, 0.8627451 , 0.8509804 ],
         ...,
         [0.84313726, 0.8509804 , 0.8392157 ],
         [0.84313726, 0.8509804 , 0.8392157 ],
         [0.85490197, 0.8627451 , 0.8509804 ]],

        [[0.8509804 , 0.85882354, 0.84705883],
         [0.84705883, 0.85490197, 0.84313726],
         [0.84705883, 0.85490197, 0.84313726],
         ...,
         [0.8509804 , 0.8509804 , 0.8509804 ],
         [0.85490197, 0.8627451 , 0.85882354],
         [0.87058824, 0.8784314 , 0.8666667 ]],

        ...,

        [[0.8627451 , 0.87058824, 0.85882354],
         [0.84313726, 0.8509804 , 0.8392157 ]

In [35]:
model_xception.predict_classes(images)

array([26, 26, 26, 26, 62, 26, 62, 26, 26, 26], dtype=int64)

In [31]:
np.argmax(model_xception.predict(images), 1)

array([26, 26, 26, 26, 62, 26, 62], dtype=int64)

In [27]:
model_xception.predict(images)

array([[3.5651643e-03, 2.3887548e-04, 2.6659430e-03, 3.0391588e-04,
        7.7882933e-04, 8.3184605e-05, 1.1570542e-03, 4.3708243e-04,
        1.6959604e-03, 8.4924661e-03, 1.2077618e-03, 5.2105282e-02,
        1.9812737e-02, 2.7237987e-04, 2.5520786e-03, 1.1560542e-02,
        3.0454653e-03, 1.7658397e-04, 1.2216343e-04, 4.0045302e-04,
        3.2380641e-02, 6.8046630e-04, 8.6587074e-04, 7.3830173e-03,
        2.0544622e-03, 1.2709297e-02, 2.9041249e-01, 2.9081434e-03,
        1.3100549e-04, 6.2592456e-04, 1.9799732e-02, 9.2905393e-05,
        2.5887995e-03, 4.1228117e-04, 9.9389371e-04, 1.0646492e-03,
        2.6749523e-04, 2.5203370e-04, 8.8576204e-04, 5.1020255e-04,
        2.4679501e-04, 4.5007057e-04, 8.4726932e-04, 9.4777271e-03,
        2.2201045e-01, 2.1934017e-04, 5.1938919e-03, 6.9408992e-04,
        1.4908448e-03, 1.1209560e-03, 1.1442811e-03, 2.8410770e-03,
        1.4088508e-04, 6.1194127e-04, 5.9482665e-03, 5.5726403e-03,
        2.8317893e-04, 4.3046084e-04, 4.5074153e

In [28]:
df_class

NameError: name 'df_class' is not defined